In [ ]:
import os

import basd
from dask.distributed import Client
from dask.distributed import LocalCluster
import xarray as xr

In [ ]:
input_dir = '../basd/data'
output_dir = '../basd/data/output'

In [ ]:
ba_output_file = 'pr_sim_fut_ba_coarse_2065-2100.nc'
basd_output_file = 'pr_sim_fut_basd_fine_2065-2100.nc'

#### Reading NetCDF file
Reading in observational precipitation data from 1979-2014 supplied in the data directory.

In [ ]:
pr_obs_hist = xr.open_mfdataset(os.path.join(input_dir, 'pr_obs-hist_fine_1979-2014.nc'), chunks={'time': 100})
pr_sim_hist = xr.open_mfdataset(os.path.join(input_dir, 'pr_sim-hist_coarse_1979-2014.nc'), chunks={'time': 100})
pr_sim_fut = xr.open_mfdataset(os.path.join(input_dir, 'pr_sim-fut_coarse_2065-2100.nc'), chunks={'time': 100})

### Running Bias Adjustment
#### Initializing our first `Adjustment` object
Here we are providing the three required data arrays and a variable name 'pr' representing the daily mean precipitation variable. Precipitation is assumed to follow a gamma distribution, and thus requires a lower bound and lower threshold parameter. We also set trend preservation mode to be 'mixed'. Thus, we set the parameter object directly specifying:

* Lower bound = 0
* Lower threshold = 0.0000011574
* Trend preservation = mixed
* Distribution = gamma

Otherwise, we are keeping the default parameter values for the bias adjustment.

In [ ]:
params = basd.Parameters(lower_bound=0, lower_threshold=0.0000011574,
                         trend_preservation='mixed',
                         distribution='gamma',
                         if_all_invalid_use=0, n_iterations=20)

#### Running Bias Adjustment
Here we run the bias adjustment for just one grid cell. We pass in a tuple with the indexes which we wish to adjust, (0,0), which in this case correspond to the coordinates 53.5 N, 12.5 E.

In [ ]:
with LocalCluster(processes=True, threads_per_worker=1) as cluster, Client(cluster) as client:
    print(client.dashboard_link)
    ba = basd.Adjustment(pr_obs_hist, pr_sim_hist, pr_sim_fut,
                         'pr', params, remap_grid=True )
    ba.adjust_bias(lat_chunk_size=1, lon_chunk_size=1,
                   file=os.path.join(output_dir, ba_output_file))

# Statistical Downscaling
We're now going to start to look at downscaling simulated data to observational data resolution using the statistical downscaling algorithm.

In [ ]:
obs_fine = xr.open_mfdataset(os.path.join(input_dir, 'pr_obs-hist_fine_1979-2014.nc'), chunks={'time': 100})
sim_coarse = xr.open_mfdataset(os.path.join(output_dir, ba_output_file), chunks={'time': 100})

In [ ]:
sd = basd.Downscaler(obs_fine, sim_coarse, 'pr', params)

In [ ]:
with LocalCluster(processes=True, threads_per_worker=1) as cluster, Client(cluster) as client:
    print(client.dashboard_link)
    sd = basd.Downscaler(obs_fine, sim_coarse, 'pr', params)
    sd.downscale(lat_chunk_size=1, lon_chunk_size=1,
                 file=os.path.join(output_dir, basd_output_file))